<a href="https://colab.research.google.com/github/kairamilanifitria/PurpleBox-Intern/blob/main/01-08%20misterworker%20scrapping%20and%20ocr%20experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Mister Worker : try -> failed (403: forbidden)

In [17]:
!pip install selenium undetected-chromedriver beautifulsoup4 pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for undetected-chromedriver: filename=undetected_chromedriver-3.5.5-py3-none-any.whl size=47048 sha256=586bbb9e4b9fe78a717666635cc89406edc9bb4f83e58452259ec3057515cc57
  Stored in directory: /root/.cache/pip/wheels/cf/a1/db/e1275b6f7259aacd6b045f8bfcb1fcbc93827a3916ba55d5b7
Successfully built undetected-chromedriver


In [26]:
!pip install playwright


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 27.1 MB/s eta 0:00:00


In [33]:
import requests
from bs4 import BeautifulSoup

# Function to simulate a search on Mister Worker website by manipulating the URL
def search_mister_worker(query):
    base_url = "https://www.misterworker.com/en"
    search_url = f"{base_url}/?search={query}"

    # Send the GET request to the search URL
    response = requests.get(search_url)
    if response.status_code == 200:
        # Parse the page content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the relevant product data (this is an example, adjust based on actual HTML structure)
        products = []
        for product in soup.find_all('div', class_='product-description'):
            name_tag = product.find('div', class_='h3 product-title')
            name = name_tag.text.strip() if name_tag else 'N/A'

            price_tag = product.find('span', class_='price')
            price = price_tag.text.strip() if price_tag else 'N/A'

            link_tag = product.find('a', class_='thumbnail product-thumbnail')
            link = link_tag['href'] if link_tag else 'N/A'

            products.append({
                'name': name,
                'price': price,
                'link': f"{base_url}{link}",
            })

        return products
    else:
        print(f"Failed to retrieve search results. Status code: {response.status_code}")
        return []

# Example: Simulating a search for 'Bosch 2-in-1 Cutting Disc'
query = "Bosch 2-in-1 Cutting Disc"
products = search_mister_worker(query)

# Display the results
for product in products:
    print(f"Name: {product['name']}, Price: {product['price']}, Link: {product['link']}")


Failed to retrieve search results. Status code: 403


## using langchain

In [2]:
!pip install --upgrade langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 22.6 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [3]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.9 MB/s eta 0:00:00


In [4]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter # Import CharacterTextSplitter

# Set the URL of the page you want to scrape
url = "https://www.misterworker.com/en/promo-mister-worker?order=product.price.asc"

# Create a WebBaseLoader instance (WebPageLoader is deprecated)
loader = WebBaseLoader(url)

# Load the webpage content
documents = loader.load()

# Optionally, split the document into smaller chunks for easier processing
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Print out the first few chunks of text to inspect
for i, chunk in enumerate(texts[:3]):
    print(f"Chunk {i+1}: {chunk.page_content[:500]}...")  # Print first 500 chars for review

Chunk 1: Attention Required! | Cloudflare


Please enable cookies.


Sorry, you have been blocked
You are unable to access misterworker.com

Why have I been blocked?
This website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data.


What can I do to resolve this?
You can email the site owner to...


In [6]:
from langchain.document_loaders import WebBaseLoader # Use WebBaseLoader
from langchain.text_splitter import CharacterTextSplitter
import requests

# Custom headers to simulate a real browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# URL of the page you want to scrape
url = "https://www.misterworker.com/en/promo-mister-worker?order=product.price.asc"

# Create the WebBaseLoader instance
loader = WebBaseLoader(url) # Remove headers argument

# Set the requests_kwargs attribute to include custom headers
loader.requests_kwargs = {'headers': headers}  # Pass headers through requests_kwargs

# Load the webpage content
documents = loader.load()

# Optionally, split the document into smaller chunks for easier processing
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

# Print out the first few chunks of text to inspect
for i, chunk in enumerate(texts[:3]):
    print(f"Chunk {i+1}: {chunk.page_content[:500]}...")  # Print first 500 chars for review

Chunk 1: Attention Required! | Cloudflare


Please enable cookies.


Sorry, you have been blocked
You are unable to access misterworker.com

Why have I been blocked?
This website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data.


What can I do to resolve this?
You can email the site owner to...


# ocr experiments

In [38]:
!pip install pytesseract pdfplumber pdf2image
!apt-get install -y tesseract-ocr
!apt-get install -y libpoppler-cpp-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libpoppler-cpp-dev is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [40]:
pdf_path = '/content/trial.pdf'

In [42]:
!apt-get install -y poppler-utils
import pdf2image
import pytesseract
from PIL import Image


pdf_path = '/content/trial.pdf'

# Convert PDF to images (one image per page)
# poppler_path is set explicitly here to locate the poppler utilities
images = pdf2image.convert_from_path(pdf_path, poppler_path="/usr/bin")

# Extract text from images using pytesseract
ocr_text = ""
for img in images:
    ocr_text += pytesseract.image_to_string(img)

print(ocr_text[:1000])  # Preview the first 1000 characters

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 2s (120 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 123859 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.5_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.5) ...
Setting up poppler-utils (22.02.0-2ubuntu0.5) ...
Processing triggers for man-db (2.10.2-1) ...
DIGITAL MEASURING TOOLS

UNIVERSAL MOUNT, LASER TARGETS, ACCESSORY

UNIVERSAL MOUNT
BM 1 Professional

Part number 0.601 015.

0.500 kg

GLL 3-80 P, GLL 2-80 P, GLL 3-50, GLL 2-50,
GLL 2-15

In [43]:
import pdfplumber

# Path to your multi-column PDF
pdf_path = '/content/trial.pdf'

with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[0]  # Get the first page of the PDF

    # Extract words along with their position (x0, top, x1, bottom)
    words = page.extract_words()

    # Display the first few words and their positions
    for word in words[:10]:
        print(word)


{'text': 'DIGITAL', 'x0': 59.5021, 'x1': 176.41679012999998, 'top': 47.110532300000045, 'doctop': 47.110532300000045, 'bottom': 75.09863229999996, 'upright': True, 'height': 27.988099999999918, 'width': 116.91469012999998, 'direction': 'ltr'}
{'text': 'MEASURING', 'x0': 183.01638411000002, 'x1': 365.05658413, 'top': 47.110532300000045, 'doctop': 47.110532300000045, 'bottom': 75.09863229999996, 'upright': True, 'height': 27.988099999999918, 'width': 182.04020001999996, 'direction': 'ltr'}
{'text': 'TOOLS', 'x0': 371.65617811, 'x1': 467.89605876999997, 'top': 47.110532300000045, 'doctop': 47.110532300000045, 'bottom': 75.09863229999996, 'upright': True, 'height': 27.988099999999918, 'width': 96.23988065999998, 'direction': 'ltr'}
{'text': 'UNIVERSAL', 'x0': 59.5021, 'x1': 131.6628168, 'top': 112.37162839999996, 'doctop': 112.37162839999996, 'bottom': 124.36642840000002, 'upright': True, 'height': 11.994800000000055, 'width': 72.1607168, 'direction': 'ltr'}
{'text': 'MOUNT,', 'x0': 134.61

In [44]:
# Sort words by their x0 (left-to-right) and then by top (top-to-bottom)
words_sorted = sorted(words, key=lambda x: (x['x0'], x['top']))

# Now you can group words into columns
columns = []
current_column = []
last_x = None

for word in words_sorted:
    if last_x is None or word['x0'] - last_x > 50:  # 50 is a threshold for column separation
        if current_column:
            columns.append(current_column)
        current_column = [word['text']]
    else:
        current_column.append(word['text'])

    last_x = word['x0']

if current_column:
    columns.append(current_column)

# Now each 'column' is a list of words in the correct order
for idx, column in enumerate(columns):
    print(f"Column {idx + 1}: {' '.join(column)}")


Column 1: 12
Column 2: DIGITAL UNIVERSAL UNIVERSAL BM LASER Laser 236 Technical (cid:51)(cid:68)(cid:85)(cid:87)(cid:3)(cid:81)(cid:88)(cid:80)(cid:69)(cid:72)(cid:85) (cid:58)(cid:72)(cid:76)(cid:74)(cid:75)(cid:87)(cid:15)(cid:3)(cid:68)(cid:83)(cid:83)(cid:85)(cid:82)(cid:91)(cid:17) (cid:54)(cid:88)(cid:76)(cid:87)(cid:68)(cid:69)(cid:79)(cid:72)(cid:3)(cid:73)(cid:82)(cid:85) (cid:38)(cid:82)(cid:80)(cid:72)(cid:86)(cid:3)(cid:70)(cid:82)(cid:80)(cid:83)(cid:79)(cid:72)(cid:87)(cid:72)(cid:3)(cid:90)(cid:76)(cid:87)(cid:75) Technical (cid:51)(cid:68)(cid:85)(cid:87)(cid:3)(cid:81)(cid:88)(cid:80)(cid:69)(cid:72)(cid:85) (cid:51)(cid:68)(cid:76)(cid:81)(cid:87) (cid:38)(cid:82)(cid:80)(cid:72)(cid:86)(cid:3)(cid:70)(cid:82)(cid:80)(cid:83)(cid:79)(cid:72)(cid:87)(cid:72)(cid:3)(cid:90)(cid:76)(cid:87)(cid:75) (cid:799) (cid:55)(cid:82)(cid:82)(cid:79)(cid:3)(cid:71)(cid:76)(cid:80)(cid:72)(cid:81)(cid:86)(cid:76)(cid:82)(cid:81)(cid:86)(cid:3)(cid:11)(cid:75)(cid:72)(cid:76)(cid:74

In [3]:
!pip install torch==2.0.1 torchvision==0.15.2 transformers==4.37.2 tiktoken==0.6.0 verovio==4.3.1 accelerate==0.28.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 881.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.

In [2]:
pip install transformers accelerate numpy Requests torch torchvision qwen-vl-utils av ipython reportlab fpdf python-docx pillow huggingface_hub

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.2 MB/s eta 0:00:00
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=b9f13a2cfdf02bba5520c26497b7da07b7d2561b0c60282952aad1071fe70efb
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [10]:
# Example usage in Google Colab
input_pdf_path = "/content/trial.png"

# Use "Qwen2VL Base" instead of "Latex OCR"
output_text = qwen_inference("Qwen2VL Base", input_pdf_path, text_input="Read the content")

# Generate the document with the output text
output_file = generate_document(input_pdf_path, output_text, file_format="pdf", font_choice="Arial", font_size=12, line_spacing=1.5, alignment="Justified", image_size="Medium")

# Print the file location
print(f"Document saved as {output_file}")

ValueError: 
paragraph text '<para>system You are a helpful assistant. user Summarize the contentUNIVERSAL MOUNT LASER TARGET BM 1 Professional Laser target (blue) Professional Profes eee ered Part number 0601015 Part number 1 608 MOO ... Weight, approx. 0.500 kg Paint tue Suitable for GLL 3-80 P, GLL 2-80 P, GLL 3-50, GLL 2-50, Tool dimensions (height) 100 mm GLL 2-15, GCL 25, GPL5, GPL 3 Professional Suitable for GRL 300 HVG Professional Peek = AO1 CE assistant The content is about a laser target called "BM 1 Professional" with a part number of</para>' caused exception error with style name=Normal Can't map determine family/bold/italic for arial

# OCR : TESSERACT (need to improve)

In [16]:
!apt-get update
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pdf2image
!apt-get install -y poppler-utils

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading

In [17]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path

# Ensure Tesseract is available
pytesseract.pytesseract.tesseract_cmd = r'/usr/bin/tesseract'

def extract_text_from_image(image_path):
    """Extract text from an image using OCR."""
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF by converting each page to an image."""
    pages = convert_from_path(pdf_path, 300)  # 300 dpi for better quality
    text = ""
    for page in pages:
        text += pytesseract.image_to_string(page)
    return text

In [20]:
# Example usage for an image
image_path = "/content/trial.png"
ocr_text_image = extract_text_from_image(image_path)
print("OCR Text from Image:", ocr_text_image)



# Save OCR result to a text file
output_file = "/content/ocr_output.txt"
with open(output_file, "w") as file:
    file.write(ocr_text_pdf)  # Or ocr_text_image for image input


OCR Text from Image: UNIVERSAL MOUNT

LASER TARGET
BM 1 Professional

Laser target (blue) Professional Profes

   

eee ered

Part number 0601015 Part number 1 608 MOO ...

Weight, approx. 0.500 kg Paint tue

Suitable for GLL 3-80 P, GLL 2-80 P, GLL 3-50, GLL 2-50, Tool dimensions (height) 100 mm
GLL 2-15, GCL 25, GPL5, GPL 3 Professional

Suitable for GRL 300 HVG Professional
Peek = AO1

CE



In [19]:
# Example usage for a PDF
pdf_path = "/content/trial.pdf"
ocr_text_pdf = extract_text_from_pdf(pdf_path)
print("OCR Text from PDF:", ocr_text_pdf)



# Save OCR result to a text file
output_file = "/content/ocr_output.txt"
with open(output_file, "w") as file:
    file.write(ocr_text_pdf)  # Or ocr_text_image for image input


OCR Text from PDF: 12

DIGITAL MEASURING TOOLS

UNIVERSAL MOUNT, LASER TARGETS, ACCESSORY

UNIVERSAL MOUNT
BM 1 Professional

_#

Bl ame)

Part number 0601 015...
Weight, approx. 0.500 kg

Suitable for GLL 3-80 P, GLL 2-80 P, GLL 3-50, GLL 2-50,
GLL 2-15, GCL 25, GPL5, GPL 3 Professional

Comes complete with Pore \ |

LASER TARGET
Laser target (red) Professional

 

Bt ae)
Part number

 

1 608 MOO ...
Paint red
Tool dimensions (height) 100 mm
Suitable for GRL 300 HV, GRL 400 H Professional
Comes complete with Pen 121 07

236 | Professional Power Tools for Trade & Industry

Comes complete with

LASER TARGET
Laser target (blue) Professional

 

BT aCe 1c)

Part number

1 608 MOO ...
Paint blue
Tool dimensions (height) 100 mm
Suitable for GRL 300 HVG Professional

Comes complete with Pn 12) 4

ACCESSORY
Battery adapter AA1 Professional

 

Br eee)

Part number
Suitable for

1 608 MOO...

GLL 3-80 CG, GLL 3-80 C, GTC 400 C,
GCL 2-50 C, GCL 2-50 CG, D-tect 120,
GIS 1000 C Professional

...

# ✅ OCR : QWEN MODEL(SUCCESS)

In [30]:
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor, TextIteratorStreamer
from qwen_vl_utils import process_vision_info
from PIL import Image
import uuid
import io
from threading import Thread
from reportlab.lib.pagesizes import A4
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Image as RLImage, Paragraph, Spacer
from reportlab.lib.units import inch
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import docx
from docx.enum.text import WD_ALIGN_PARAGRAPH

# Load the model and processor
model_name = "Qwen/Qwen2-VL-2B-Instruct"
model = Qwen2VLForConditionalGeneration.from_pretrained(model_name, trust_remote_code=True, torch_dtype=torch.float16).to("cuda").eval()
processor = AutoProcessor.from_pretrained(model_name, trust_remote_code=True)

image_extensions = Image.registered_extensions()

def identify_and_save_blob(blob_path):
    """Identifies if the blob is an image and saves it."""
    try:
        with open(blob_path, 'rb') as file:
            blob_content = file.read()
            try:
                Image.open(io.BytesIO(blob_content)).verify()  # Check if it's a valid image
                extension = ".png"  # Default to PNG for saving
                media_type = "image"
            except (IOError, SyntaxError):
                raise ValueError("Unsupported media type. Please upload a valid image.")

            filename = f"temp_{uuid.uuid4()}_media{extension}"
            with open(filename, "wb") as f:
                f.write(blob_content)

            return filename, media_type

    except FileNotFoundError:
        raise ValueError(f"The file {blob_path} was not found.")
    except Exception as e:
        raise ValueError(f"An error occurred while processing the file: {e}")

def qwen_inference(media_input, text_input=None):
    """Handles inference for the selected model."""
    if isinstance(media_input, str):
        media_path = media_input
        if media_path.endswith(tuple([i for i in image_extensions.keys()])):
            media_type = "image"
        else:
            try:
                media_path, media_type = identify_and_save_blob(media_input)
            except Exception as e:
                raise ValueError("Unsupported media type. Please upload a valid image.")

    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": media_type,
                    media_type: media_path
                },
                {"type": "text", "text": text_input},
            ],
        }
    ]

    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, _ = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    ).to("cuda")

    # Run the model in a separate thread
    streamer = TextIteratorStreamer(
        processor.tokenizer, skip_prompt=True, skip_special_tokens=True
    )
    generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=1024)

    thread = Thread(target=model.generate, kwargs=generation_kwargs)
    thread.start()

    buffer = ""
    for new_text in streamer:
        buffer += new_text
        # Remove <|im_end|> or similar tokens from the output
        buffer = buffer.replace("<|im_end|>", "")
        yield buffer

def format_plain_text(output_text):
    """Formats the output text as plain text without LaTeX delimiters."""
    # Remove LaTeX delimiters and convert to plain text
    plain_text = output_text.replace("\\(", "").replace("\\)", "").replace("\\[", "").replace("\\]", "")
    return plain_text

def generate_pdf(media_path, plain_text, font_choice, font_size, line_spacing, alignment, image_size):
    """Generates a PDF document."""
    filename = f"output_{uuid.uuid4()}.pdf"
    doc = SimpleDocTemplate(
        filename,
        pagesize=A4,
        rightMargin=inch,
        leftMargin=inch,
        topMargin=inch,
        bottomMargin=inch
    )
    styles = getSampleStyleSheet()
    styles["Normal"].fontName = font_choice
    styles["Normal"].fontSize = int(font_size)
    styles["Normal"].leading = int(font_size) * line_spacing
    styles["Normal"].alignment = {
        "Left": 0,
        "Center": 1,
        "Right": 2,
        "Justified": 4
    }[alignment]

    # Register font
    font_path = f"font/{font_choice}"
    pdfmetrics.registerFont(TTFont(font_choice, font_path))

    story = []

    # Add image with size adjustment
    image_sizes = {
        "Small": (200, 200),
        "Medium": (400, 400),
        "Large": (600, 600)
    }
    img = RLImage(media_path, width=image_sizes[image_size][0], height=image_sizes[image_size][1])
    story.append(img)
    story.append(Spacer(1, 12))

    # Add plain text output
    text = Paragraph(plain_text, styles["Normal"])
    story.append(text)

    doc.build(story)
    return filename

def generate_docx(media_path, plain_text, font_choice, font_size, line_spacing, alignment, image_size):
    """Generates a DOCX document."""
    filename = f"output_{uuid.uuid4()}.docx"
    doc = docx.Document()

    # Add image with size adjustment
    image_sizes = {
        "Small": docx.shared.Inches(2),
        "Medium": docx.shared.Inches(4),
        "Large": docx.shared.Inches(6)
    }
    doc.add_picture(media_path, width=image_sizes[image_size])
    doc.add_paragraph()

    # Add plain text output
    paragraph = doc.add_paragraph()
    paragraph.paragraph_format.line_spacing = line_spacing
    paragraph.paragraph_format.alignment = {
        "Left": WD_ALIGN_PARAGRAPH.LEFT,
        "Center": WD_ALIGN_PARAGRAPH.CENTER,
        "Right": WD_ALIGN_PARAGRAPH.RIGHT,
        "Justified": WD_ALIGN_PARAGRAPH.JUSTIFY
    }[alignment]
    run = paragraph.add_run(plain_text)
    run.font.name = font_choice
    run.font.size = docx.shared.Pt(int(font_size))

    doc.save(filename)
    return filename

The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [31]:
# Example usage
media_input = "/content/trial.png"
text_input = "Summarize the content of the image"

# Generate raw output from model
output = list(qwen_inference(media_input, text_input))
print(output)

['', 'The ', 'The image ', 'The image is ', 'The image is a ', 'The image is a comparison ', 'The image is a comparison between ', 'The image is a comparison between two ', 'The image is a comparison between two ', 'The image is a comparison between two products: ', 'The image is a comparison between two products: a ', 'The image is a comparison between two products: a Universal ', 'The image is a comparison between two products: a Universal Mount ', 'The image is a comparison between two products: a Universal Mount BM ', 'The image is a comparison between two products: a Universal Mount BM ', 'The image is a comparison between two products: a Universal Mount BM 1 ', 'The image is a comparison between two products: a Universal Mount BM 1 Professional ', 'The image is a comparison between two products: a Universal Mount BM 1 Professional and ', 'The image is a comparison between two products: a Universal Mount BM 1 Professional and a ', 'The image is a comparison between two products: a

In [33]:
# Format output as plain text
plain_text = format_plain_text(output[-1])
print(plain_text)

The image is a comparison between two products: a Universal Mount BM 1 Professional and a Laser Target Laser target (blue) Professional. Here is a summary of the content:

### Universal Mount BM 1 Professional
- **Part Number**: 0 601 015
- **Weight**: Approx. 0.500 kg
- **Suitable for**: GLL 3-80 P, GLL 2-80 P, GLL 3-50, GLL 2-50, GLL 2-15, GCL 25, GPL5, GPL 3 Professional
- **Comes complete with**: A01

### Laser Target Laser target (blue) Professional
- **Part Number**: 1 608 M00
- **Paint**: Blue
- **Tool dimensions (height)**: 100 mm
- **Suitable for**: GRL 300 HVG Professional
- **Comes complete with**: 05K

### Technical Data
- **Universal Mount BM 1 Professional**
  - **Part Number**: 0 601 015
  - **Weight**: Approx. 0.500 kg
  - **Suitable for**: GLL 3-80 P, GLL 2-80 P, GLL 3-50, GLL 2-50, GLL 2-15, GCL 25, GPL5, GPL 3 Professional
  - **Comes complete with**: A01

- **Laser Target Laser target (blue) Professional**
  - **Part Number**: 1 608 M00
  - **Paint**: Blue
  - **Too